In [ ]:
%load_ext autoreload
%autoreload 2

import optuna
import json
import shutil
import matplotlib.pyplot as pl
%matplotlib inline
import math
import random
import os
import sys
import datasets
import torch
import transformers
sys.path.append('..')
from model_manager import ModelManager
from model_utils import train_model, split_data
from config import Config

In [ ]:
study_name = "AWSW Best Learning Params"
storage_url = f"sqlite:///{study_name}.db"
study = optuna.create_study(study_name=study_name, storage=storage_url, load_if_exists=True)
datasets.logging.set_verbosity(datasets.logging.ERROR)
# Tell pytorch to run this model on the GPU.
device_name = "cuda:0" if torch.cuda.is_available() else "cpu"
# device_name = "cpu"
device = torch.device(device_name)

In [ ]:
def objective(trial):
    config = {
        "model_name": "EleutherAI/gpt-neo-125M",
        "lr": 6e-4,
        "warmup_factor": 2,
        "scheduler": "polynomial_decay_schedule_with_warmup",
        "lr_end": 2e-6,
        "power": 0.6,
        "freeze_from_steps": 10,
        "seed": 80085,
        "num_epoch": 5,
        "to_freeze_count": 120,
        "model_folder": os.path.join(Config.work_dir, "models", "awsw_hyperopt")
    }
    config['freeze_from_steps'] = trial.suggest_int('freeze_from_steps', -1, 200)
    config['to_freeze_count'] = trial.suggest_int('to_freeze_count', 10, 120)
    train_results = {}
    # There's sadly no way to train without saving the model in Huggingface so I'll delete it each time we begin training.
    if os.path.isdir(config['model_folder']): 
        shutil.rmtree(config['model_folder'])
    transformers.logging.set_verbosity_error()
    train_model(config, train_results, device = device)
    return train_results['loss_history'][-1]

study.optimize(objective, n_trials=100)

In [ ]:
study.best_params

In [ ]:
!optuna-dashboard --host 0.0.0.0 "$storage_url"